In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import tgt
import os
import numpy as np
from tqdm import tqdm
import torchaudio
import librosa
import glob

sys.path.append("/mntcephfs/lab_data/shoinoue/Models/trained_models/vocos/vocos16k_noncausal_tealab/")
from vocos16k_inference import Vocos

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram
from sound import play_audio

from vocos.feature_extractors import MelSpectrogramFeatures, EncodecFeatures

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [9]:
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"
fs = 24000

melgen = MelSpectrogramFeatures(sample_rate=fs)
speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*/*")]
speakers.sort()

In [97]:
save = True

files = []
for spk in tqdm(speakers):
    files += glob.glob(dataset_dir + f"*/{spk}/*/*.wav")
files.sort()
for path in tqdm(files):
    y, sr = torchaudio.load(path)
    if fs!=sr:
        if y.size(0) > 1:  # mix to mono
            y = y.mean(dim=0, keepdim=True)
        y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=fs)
    mel = melgen(y[0])
    savepath = feat_base_dir + "/".join(path.split("/")[-4:])[:-4] + ".npy"
    if save:
        np.save(savepath, mel)

100%|██████████| 160267/160267 [28:47<00:00, 92.78it/s] 


# Normalize

In [98]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

In [99]:
scaler_name = "LibriTTS-R"

In [100]:
scaler = StandardScaler()
files = []
for spk in tqdm(speakers):
    files += glob.glob(feat_base_dir + f"train*/{spk}/*/*[0-9].npy")
files.sort()
for path in tqdm(files):
    mel = np.load(path)
    scaler.partial_fit(mel.T)
            
scaler_filename = f"ckpts/scalers/{scaler_name}.save"
joblib.dump(scaler, scaler_filename) 
# a = joblib.load(scaler_filename)

100%|██████████| 149694/149694 [02:17<00:00, 1086.71it/s]


['ckpts/scalers/LibriTTS-R.save']